In [1]:
import soundfile
import torch
import matplotlib.pyplot as plt
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text

In [ ]:
def save_image(image_list, name, n_layers, n_heads):
    rows = n_layers
    cols = n_heads
    fig_saved_dir = './exp/feature_images/'

    for j in range(rows):
        fig = plt.figure(figsize=(100,100))
        axes = []
        for i in range(cols):
            img = image_list[j*cols+i]
            axes.append(fig.add_subplot(1, cols, i+1))
            plt.imshow(img)
            plt.axis('off')

        print('process {0} layer images....'.format(j))
        plt.savefig(fig_saved_dir + 'audio' + str(audio_num) + '_' + name +'_layer{0}_attention.png'.format(j),
                    bbox_inches='tight',
                    dpi=100)

    # Use show when you want to show your attention image while extraction
    # plt.show() 

# Set test wav for attention image extraction
TEST_DATA_PATH = "./data/dev_clean"
WAV_LIST_PATH = TEST_DATA_PATH + "/wav.scp"

file_name_list = []
audio_num = 1 # selelct one of the wav in file_name_list

with open(WAV_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        num, name = line.split(' ')
        file_name_list.append(name[:-1])

speech, rate = soundfile.read(file_name_list[audio_num])

# Prepare model
d = ModelDownloader()

ASR_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_train_asr_transformer_e18_raw_bpe_sp/54epoch.pth"
LM_MODEL_PATH = "/home_data/jmpark/espnet/tools/anaconda/envs/espnet_1.7.1/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/lm_train_lm_adam_bpe/17epoch.pth"

speech2text = Speech2Text(
    **d.download_and_unpack('Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best'),
    # Decoding parameters are not included in the model file
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=1,
    ctc_weight=0.4,
    lm_weight=0.6,
    penalty=0.0,
    nbest=1
)
# Add register hook for in encoder layers.
net = speech2text.asr_model

In [ ]:
saved_encoder_self_attn_images = []

def attn_encoder(self, input_tensor, output_tensor):
    saved_encoder_self_attn_images.append(output_tensor)

for name, parameter in net.named_modules():
    for i in range(18): 
        if 'encoder.encoders.'+ str(i) +'.self_attn' == name:
            print(name)
            print(parameter)
            parameter.register_forward_hook(attn_encoder)

In [ ]:
saved_decoder_self_attn_images = []
saved_decoder_self_attn_images_in = []
def attn_decoder(self, input_tensor, output_tensor):
    saved_decoder_self_attn_images_in.append(input_tensor)
    saved_decoder_self_attn_images.append(output_tensor)
    # cols = output_tensor[1].shape[1]

    # for i in range(cols):
    #     img = output_tensor[1][0,i,:]
    #     saved_decoder_self_attn_images.append(img)

for name, parameter in net.named_modules():
    for i in range(6): 
        if 'decoder.decoders.'+ str(i) +'.self_attn' == name:
            print(name)
            print(parameter)
            parameter.register_forward_hook(attn_decoder)

In [ ]:
saved_encoder_self_attn_images = []
saved_decoder_self_attn_images = []
saved_decoder_self_attn_images_in = []
out = speech2text(speech)

In [ ]:
print(len(saved_decoder_self_attn_images))
print(len(saved_decoder_self_attn_images_in))
print(len(saved_encoder_self_attn_images))

In [ ]:
en_output_tensor = saved_encoder_self_attn_images[10]
en_cols = en_output_tensor[1].size()
print(len(en_output_tensor))
print(en_output_tensor[0].shape)
print(en_output_tensor[1].shape)
print(en_cols)

for i in range(8):
    img = en_output_tensor[1]
    print(img.shape)

In [ ]:
de_output_tensor = saved_decoder_self_attn_images[5]
print(de_output_tensor[0].size())
de_cols = de_output_tensor[1].size(1)
print(de_cols)

for i in range(de_cols):
    img = de_output_tensor[1]
    print(img.shape)

In [ ]:
de_output_tensor = saved_decoder_self_attn_images_in[24]
print(de_output_tensor[2])
print(de_output_tensor[1])
de_cols = de_output_tensor[1].size(1)
print(de_cols)

for i in range(de_cols):
    img = de_output_tensor[1]
    print(img.shape)

In [ ]:
# save_image(saved_decoder_self_attn_images, 'decoder_self_attn', 6, 8)